<a href="https://colab.research.google.com/github/armandossrecife/my_validation3/blob/main/my_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup do Ambiente de Análise

In [191]:
!rm -rf *.log
!rm -rf *.txt
!rm -rf *.xlsx
!rm -rf my_issues
!rm -rf cassandra

In [192]:
print('Install Pydriller.')
!pip install pydriller > install_pydriller.log
print('Install gitpython.')
!pip3 install gitpython > install_gitpython.log
print('Install Jira Python lib.')
!pip install jira > install_jira_python.log
print('Install SQLite in Linux')
!sudo apt install -y sqlite3 > install_sqlite.log
print('All depenpencies installed!')
!cat *.log > install.log
print('Details in install.log')

Install Pydriller.
Install gitpython.
Install Jira Python lib.
Install SQLite in Linux


All depenpencies installed!
Details in install.log


## Importa bibliotecas necessárias

In [193]:
import os
from datetime import datetime, timedelta
import tqdm
from pydriller import Repository
from jira import JIRA
import pandas as pd
import re
import scipy.stats as stats
import random
import shutil
import time

## Variáveis Globais

In [220]:
my_repository = 'cassandra'
url_to_repository = 'https://github.com/apache/cassandra.git'
os.environ['MY_REPOSITORY'] = url_to_repository

JIRA_SERVER = 'https://issues.apache.org/jira'
ISSUE_TRACKER_PROJECT = 'CASSANDRA'

# Credentials
os.environ['USERNAME'] = '?'
os.environ['PASSWORD'] = '?'
username = os.environ.get('USERNAME')
password = os.environ.get('PASSWORD')

lista_arquivos_criticos = ['StorageService.java', 'ColumnFamilyStore.java','DatabaseDescriptor.java','CompactionManager.java','StorageProxy.java','SSTableReader.java','Config.java','CassandraDaemon.java','SelectStatement.java','SinglePartitionReadCommand.java','NodeProbe.java','MessagingService.java']

## Clona o Repositório

In [195]:
print(f'Clona o repositório : {url_to_repository}')
!git clone $MY_REPOSITORY

Clona o repositório : https://github.com/apache/cassandra.git
Cloning into 'cassandra'...
remote: Enumerating objects: 405556, done.
remote: Counting objects: 100% (1119/1119), done.
remote: Compressing objects: 100% (737/737), done.
remote: Total 405556 (delta 391), reused 765 (delta 235), pack-reused 404437
Receiving objects: 100% (405556/405556), 406.33 MiB | 19.72 MiB/s, done.
Resolving deltas: 100% (236498/236498), done.
Updating files: 100% (6212/6212), done.


## Funções de apoio

In [196]:
def get_all_commits_by_range(initial_date, final_date, repository_name):
  """Extracts information from a date range of commits
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commit's date, commit's line, commit's file, a list of files modified in the commit, a list of all diffs from modified files
  """
  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress commit anlysis"):
      list_of_modified_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_m_diff = {}
        if m is not None:
          list_of_modified_files.append(m.filename)
          dict_m_diff[m.filename] = m.diff
          list_dict_of_diff_modified_files.append(dict_m_diff)
      data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
      element = commit.msg, data_commit, commit.lines, commit.files, list_of_modified_files, list_dict_of_diff_modified_files
      dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

def get_commits_by_range_and_critical_files(initial_date, final_date, critical_files, repository_name):
  """Extracts information from a date range of commits, focusing on critical files.
  Args:
      initial_date: inicial date of commits
      final_date: final date of commits
      critical_files: A list of critical file paths. ex: ['StorageService.java', 'ColumnFamilyStore.java']
  Returns:
        A dictionary of commits, where the keys are the commit hashes and the
        values are tuples containing the commit message, commits's date, commit's line, commit's file, a list of critical files modified in the commit, a list_of_modified_files, a dictionary of diff [filename]:filename.diff, a list of all diffs from modified files
        commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
  """

  dict_commit_modified_files = {}
  print('Wait...')
  my_traverser_commits = Repository(repository_name, since=initial_date, to=final_date).traverse_commits()
  total_commits = len(list(my_traverser_commits))
  try:
    for commit in tqdm.tqdm(Repository(repository_name, since=initial_date, to=final_date).traverse_commits(), total=total_commits, desc="Progress critical files commit anlysis"):
      list_of_critical_files_modified = []
      list_of_modified_files = []
      list_dict_of_diff_files = []
      list_dict_of_diff_modified_files = []
      for m in commit.modified_files:
        dict_diff_file = {}
        list_of_modified_files.append(m.filename)
        list_dict_of_diff_modified_files.append(m.diff)
        if m.filename in critical_files:
          list_of_critical_files_modified.append(m.filename)
          dict_diff_file[m.filename] = m.diff
          list_dict_of_diff_files.append(dict_diff_file)
      if len(list_of_critical_files_modified) > 0:
        data_commit = str(commit.committer_date.day) + '/' + str(commit.committer_date.month) + '/' + str(commit.committer_date.year)
        element = commit.msg, data_commit, commit.lines, commit.files, list_of_critical_files_modified, list_of_modified_files, list_dict_of_diff_files, list_dict_of_diff_modified_files
        dict_commit_modified_files[commit.hash] = element
  except Exception as ex:
    print(f'Erro during travesse commits: {str(ex)}')
  return dict_commit_modified_files, total_commits

In [197]:
class JiraIssue:
  def __init__(self, key, summary, issue_type, status, priority, description, comments):
    self.key = key
    self.summary = summary
    self.issue_type = issue_type
    self.status = status
    self.priority = priority
    self.description = description
    self.comments = comments

  def get_comments(self):
    return self.comments

  def __str__(self):
    return (f'Key: {self.key}, Summary: {self.summary}, Type: {self.issue_type}, Status: {self.status}')

class JiraIssues:
  def __init__(self,project, issues):
    self.project = project
    self.issues = issues

  def add_issue(self, issue):
    self.issues.append(issue)

  def get_issues(self) -> list:
    return self.issues

  def update_issues(self, issues):
    self.issues = issues

  def __str__(self):
    str_issues = ""
    for issue in self.get_issues():
      str_issues = str_issues + str(issue)
      str_issues = str_issues + ', '
    str_issues = '[' + str_issues + ']'
    return (f'Project: {self.project}, Qdt of issues: {len(self.issues)}, Issues: {str_issues}')

# Classe de utilidades para manipular o servidor Jira
class JiraUtils:
  def __init__(self, project, jira_instance):
    self.project = project
    self.jira_jira_instance = jira_instance

  def generate_intervals_between_dates(self, date1: tuple, date2: tuple, distance=120) -> list:
    start_date = datetime(date1[0], date1[1], date1[2])
    end_date = datetime(date2[0], date2[1], date2[2])
    interval_days = distance
    # Initialize a list to store the intervals
    intervals = []
    # Initialize the current date as the start date
    current_date = start_date
    # Loop to generate intervals until the current date is less than or equal to the end date
    while current_date < end_date:
        interval = (current_date, current_date + timedelta(days=interval_days - 1))
        intervals.append(interval)
        current_date += timedelta(days=interval_days)
    return intervals

  def convert_interval_dates(self, dates: list) -> list:
    list_interval_dates = []
    for each in dates:
      date1 = each[0]
      # Convert the date to a string in the format "YYYY/MM/DD".
      str_date1 = date1.strftime("%Y/%m/%d")
      date2 = each[1]
      str_date2 = date2.strftime("%Y/%m/%d")
      elemento = str_date1, str_date2
      list_interval_dates.append(elemento)
    return list_interval_dates

  def generate_list_of_sentences(self, dates: list) -> list:
    lista_sentencas = []
    for each in dates:
      str_date1 = each[0].strftime("%Y/%m/%d")
      str_date2 = each[1].strftime("%Y/%m/%d")
      sentenca = f'project={self.project.upper()} and created>="{str_date1}" and created<="{str_date2}"'
      lista_sentencas.append(sentenca)
    return lista_sentencas

  def get_list_of_block_issues_by_dates(self,date1, date2, distance=120) -> list:
    print('Aguarde...')
    t1 = datetime.now()
    list_of_dates = self.generate_intervals_between_dates(date1,date2,distance)
    lista_sentencas = self.generate_list_of_sentences(list_of_dates)
    lista_bloco_issues_by_date = []
    total_items = len(lista_sentencas)
    i = 0
    iterable_lista_sentencas = tqdm.tqdm(lista_sentencas, total=total_items)
    for each in iterable_lista_sentencas:
      issues_by_date_temp = self.jira_jira_instance.search_issues(each,maxResults=1000)
      print(f'Range: {each}, qtd issues: {len(issues_by_date_temp)}')
      lista_bloco_issues_by_date.append(issues_by_date_temp)
      percentage = (i + 1) / total_items * 100
      iterable_lista_sentencas.set_description(f"Progress Message Analysis")
    i += 1
    t2 = datetime.now()
    print(t2)
    print(f'Tempo da consulta: {t2-t1}')
    return lista_bloco_issues_by_date

  def concatenate_block_of_issues(self,block_of_issues):
    concatenated_list = [item for sublist in block_of_issues for item in sublist]
    print(f'Total de issues recuperados: {len(concatenated_list)}')
    return concatenated_list

In [198]:
def analyze_jira_all_issues(project, all_issues):
    """Extracts all issues from Issue Tracker
        Args:
          project: Issue Tracker Project name, example: CASSANDRA
          all_issues: a block of issues
        Returns:
          A list of JiraIssues
    """
    # Create an instance of JiraIssues to manage all issues
    my_all_issues = JiraIssues(project, [])
    total_items = len(all_issues)

    # Iterate through the fetched issues
    for issue in tqdm.tqdm(all_issues, total=total_items, desc='Progress jira all issues analysis'):
        issue_key = issue.key
        issue_summary = issue.fields.summary
        issue_description = issue.fields.description
        issue_comments = [comment.body for comment in issue.fields.comment.comments]

        # Check for SATD keywords in the issue's summary, description, and comments
        if issue_summary is None:
            issue_summary = ""
        if issue_description is None:
            issue_description = ""
        if issue_comments is None:
            issue_comments = ""

        issue_type = issue.fields.issuetype.name
        issue_status = issue.fields.status.name
        issue_priority = issue.fields.priority.name

        # Create a JiraIssue instance for the SATD issue
        my_issue = JiraIssue(issue_key, issue_summary, issue_type, issue_status, issue_priority, issue_description, issue_comments)

        my_all_issues.add_issue(my_issue)

    return my_all_issues

In [199]:
def convert_issues_to_dataframe(all_real_issues):
  l_issue_key_aux, l_issue_type_aux, l_issue_summary_aux, l_issue_description_aux, l_issue_status_aux, l_issue_priority_aux, l_issue_comments_aux = [], [], [], [], [], [], []

  for issue in all_real_issues.get_issues():
    l_issue_key_aux.append(issue.key)
    l_issue_type_aux.append(issue.issue_type)
    l_issue_summary_aux.append(issue.summary)
    l_issue_description_aux.append(issue.description)
    l_issue_status_aux.append(issue.status)
    l_issue_priority_aux.append(issue.priority)
    texto_aux = ""
    for item in issue.get_comments():
      texto_aux = texto_aux + str(item) + "\n"
    l_issue_comments_aux.append(texto_aux)

  dict_all_reall_issues_in_commits_detailed = {
  'issue_key': l_issue_key_aux,
  'issue_type':l_issue_type_aux,
  'status':l_issue_status_aux,
  'priority':l_issue_priority_aux,
  'summary':l_issue_summary_aux,
  'description':l_issue_description_aux,
  'comments':l_issue_comments_aux
  }

  df_all_reall_issues_in_commits_detailed = pd.DataFrame(dict_all_reall_issues_in_commits_detailed)
  return df_all_reall_issues_in_commits_detailed

def convert_commits_to_dataframe(dict_of_commits):
  '''
  v[0] = commit.msg,
  v[1] = data_commit,
  v[2] = commit.lines,
  v[3] = commit.files,
  v[4] = list_of_critical_files_modified,
  v[5] = list_of_modified_files,
  v[6] = list_dict_of_diff_files,
  v[7] = list_dict_of_diff_modified_files
  '''
  l_commit_hash, l_commit_msg, l_commit_data, l_commit_lines, l_commit_files, l_commit_critical_files, l_commit_modified_fies, l_commit_diff_files, l_commit_diff_modified_files = [], [], [], [], [], [], [], [], []
  for k, v in dict_of_commits.items():
    l_commit_hash.append(k)
    l_commit_msg.append(v[0])
    l_commit_data.append(v[1])
    l_commit_lines.append(v[2])
    l_commit_files.append(v[3])
    l_commit_critical_files.append(v[4])
    l_commit_modified_fies.append(v[5])
    l_commit_diff_files.append(v[6])
    l_commit_diff_modified_files.append(v[7])

  dict_of_commits_aux = {
      'hash': l_commit_hash,
      'msg': l_commit_msg,
      'date': l_commit_data,
      'lines': l_commit_lines,
      'files': l_commit_files,
      'critical_files': l_commit_critical_files,
      'modified_files': l_commit_modified_fies,
      'diff_files': l_commit_diff_files,
      'diff_files_modified_files': l_commit_diff_modified_files
  }

  df_commits = pd.DataFrame(dict_of_commits_aux)
  return df_commits

In [200]:
def find_issues_id_by_project(input_string: str, project: str) -> list[str]:
    """Finds all Cassandra issue ID patterns in the input string.
    Args:
        input_string: The input string.
        project: The pattern related to project name, for example: CASSANDRA project name
    Returns:
        A list of project issue IDs, if found; otherwise, an empty list.
    """
    # Try to find all Cassandra issue ID patterns in the input string
    matches = re.findall(r"({0}-\d+)".format(project), input_string)
    # Return an empty list if no matches are found
    if not matches:
        return []
    # Convert the list of matches to a set to remove duplicates
    set_matches = set(matches)
    # Convert the set of matches back to a list
    list_unique_matches = list(set_matches)

    # Return the list of matched Cassandra issue IDs
    return list_unique_matches

def get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files):
  dict_issues_in_commits = {}
  for index in df_commits_with_critical_files.index:
    l_issues_in_commit = find_issues_id_by_project(input_string=df_commits_with_critical_files.msg[index], project=ISSUE_TRACKER_PROJECT)
    if len(l_issues_in_commit) > 0:
      commit_hash = df_commits_with_critical_files.hash[index]
      dict_issues_in_commits[commit_hash] = l_issues_in_commit

  list_issue_commits, list_issue_issues = [], []
  for k, v in dict_issues_in_commits.items():
    list_issue_commits.append(k)
    for issue in v:
      if issue not in list_issue_issues:
        list_issue_issues.append(issue)

  df_aux  = df_all_reall_issues_in_commits_detailed.copy()
  df_issues_in_commits_with_critical_classes = df_aux[df_aux['issue_key'].isin(list_issue_issues)]

  return dict_issues_in_commits, df_issues_in_commits_with_critical_classes

def calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size):
    # Calculate the Z-score for the given confidence level
    z_score = stats.norm.ppf(1 - (1 - confidence_level) / 2)

    # Calculate the sample size formula
    sample_size = ((z_score**2) * population_proportion * (1 - population_proportion)) / (margin_of_error**2)

    # Adjust for finite population
    if population_size:
        sample_size = sample_size / (1 + ((sample_size - 1) / population_size))

    return int(sample_size)

In [201]:
def get_max_n_chars(text, max_n):
  text_length = len(text)
  if text_length <= max_n:
    return text
  else:
    return text[:max_n]

def create_new_file(filename, dir_name, issue_type, summary, description, status, comments):
  try:
    filename = dir_name + '/' + filename

    if issue_type is None:
      issue_type = ''
    if summary is None:
      summary = ''
    if description is None:
      description = ''
    if status is None:
      status = ''
    if comments is None:
      comments = ''

    with open(filename, mode='w') as f_issue:
      f_issue.write(f'issue_type: {issue_type} \n')
      f_issue.write(f'summary: {summary} \n')
      f_issue.write(f'description: {get_max_n_chars(text=description, max_n=1000)} \n')
      f_issue.write(f'status: {status} \n')
      f_issue.write(f'comments: {get_max_n_chars(text=comments, max_n=4000)} \n')
    print(f'File {filename} created with success!')

  except Exception as ex:
    print(f'Erro ao criar arquivo: {str(ex)}')


In [202]:
# Seleciona randomicamente os issues para inspeção
def select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date='02/11/2023'):
  lista_issues_inspecao = []
  dict_issues_para_inspecao = {}
  list_issue_key = df_issues_in_commits_with_critical_classes.issue_key.to_list()
  list_issue_key = list(set(list_issue_key))
  sample_issues = random.choices(list_issue_key, k=sample_size)
  dict_issues_para_inspecao['02/11/2023'] = sample_issues
  print(f'{len(sample_issues)} para inspeção manual')

  date_file_name = my_date.split('/')
  date_file_name = date_file_name[0] + date_file_name[1] + date_file_name[2]
  file_name = 'issues_inspecao_' + date_file_name + '.txt'
  with open(file_name, mode='w') as f_temp:
    for v in dict_issues_para_inspecao[my_date]:
      elemento = v + ','
      f_temp.write(elemento)
  print(f'Relação de Issues salvos em {my_date} para inspeção.')
  return sample_issues

# Gera os arquivos .txt de cada issue selecionado para inspeção
def generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes):
  contador = 0
  my_dir_name = 'my_issues'
  if not os.path.exists(my_dir_name):
    os.makedirs(my_dir_name)

  total_of_issues = df_issues_in_commits_with_critical_classes.shape[0]
  for index in tqdm.tqdm(df_issues_in_commits_with_critical_classes.index, total=total_of_issues, desc='Analyzing issues'):
    for issue in sample_issues:
      if df_issues_in_commits_with_critical_classes.issue_key[index] == issue:
        create_new_file(filename=df_issues_in_commits_with_critical_classes.issue_key[index], dir_name=my_dir_name, issue_type=df_issues_in_commits_with_critical_classes.issue_type[index], summary=df_issues_in_commits_with_critical_classes.summary[index], description=df_issues_in_commits_with_critical_classes.description[index], status=df_issues_in_commits_with_critical_classes.status[index], comments=df_issues_in_commits_with_critical_classes.comments[index])
        contador += 1
  print(f'Foram criados {contador} arquivos para inspeção')

# 1. Extrai commits de um repositório de código

In [203]:
start_date = datetime(2023, 1, 1, 0, 0, 0)
end_date = datetime(2023, 10, 4, 0, 0, 0)
extract_all_commits = get_all_commits_by_range(initial_date=start_date, final_date=end_date, repository_name=my_repository)

all_commits = extract_all_commits[0]
total_all_commits = extract_all_commits[1]

print(f'Total de commits extraídos: {total_all_commits}')

Wait...


Progress commit anlysis: 100%|██████████| 1014/1014 [00:44<00:00, 22.84it/s]

Total de commits extraídos: 1014


## Filtra os commits com classes críticas

In [204]:
print(f'Analisa a faixa de commits entre: initial_date={str(start_date)}, final_date={str(end_date)}')
print('Registra apenas os commits que contem pelo menos um arquivo crítico')
filter_commits_with_critical_files = get_commits_by_range_and_critical_files(initial_date=start_date, final_date=end_date, critical_files=lista_arquivos_criticos, repository_name=my_repository)
commits_with_critical_files = filter_commits_with_critical_files[0]
total_of_commits_with_critical_files = filter_commits_with_critical_files[1]

print(f'Total de commits com classes críticas: {total_of_commits_with_critical_files}')

Analisa a faixa de commits entre: initial_date=2023-01-01 00:00:00, final_date=2023-10-04 00:00:00
Registra apenas os commits que contem pelo menos um arquivo crítico
Wait...


Progress critical files commit anlysis: 100%|██████████| 1014/1014 [00:16<00:00, 62.33it/s]

Total de commits com classes críticas: 1014


### Converte os commits com classes críticas para dataframe

In [205]:
df_commits_with_critical_files = convert_commits_to_dataframe(dict_of_commits=commits_with_critical_files)
df_commits_with_critical_files

,hash,msg,date,lines,files,critical_files,modified_files,diff_files,diff_files_modified_files
0,e936b2cc1ba7f525c636de5f9fb1780ca70f1762,fix serialization error in getsstables --show-...,11/1/2023,29,5,"[ColumnFamilyStore.java, NodeProbe.java]","[CHANGES.txt, ColumnFamilyStore.java, ColumnFa...","[{'ColumnFamilyStore.java': '@@ -64,12 +64,10 ...","[@@ -1,4 +1,5 @@\n 4.2\n+ * Fix serialization ..."
1,b8a87abba47441d97b3c85a0473d02919e1c071b,Do not leak 2015 synthetic memtable Epoch\n\np...,12/1/2023,50,5,[SinglePartitionReadCommand.java],"[build.xml, Memtable.java, SinglePartitionRead...","[{'SinglePartitionReadCommand.java': '@@ -713,...","[@@ -1227,6 +1227,8 @@\n <!-- disable ..."
2,5be1038c5d38af32d3cbb0545d867f21304f3a46,Streaming progress virtual table lock contenti...,12/1/2023,512,24,"[Config.java, DatabaseDescriptor.java]","[CHANGES.txt, cassandra.yaml, Config.java, Dat...","[{'Config.java': '@@ -852,6 +852,9 @@ public c...","[@@ -1,5 +1,6 @@\n 4.1.1\n-* Fix perpetual loa..."
3,36a99bcdcc65eb2b3bb708f67e1bd75d35359f7f,Remove mocking in InternalNodeProbe spying on ...,17/1/2023,78,4,[StorageService.java],"[CHANGES.txt, StorageService.java, Instance.ja...","[{'StorageService.java': '@@ -231,6 +231,9 @@ ...","[@@ -1,4 +1,5 @@\n 4.2\n+ * Remove mocking in ..."
4,035705f49464a4854482e1f1280a7af45f7f0203,Print friendly error when nodetool attempts to...,23/1/2023,89,23,[StorageService.java],"[CHANGES.txt, StorageService.java, NodeTool.ja...","[{'StorageService.java': '@@ -381,12 +381,16 @...","[@@ -1,4 +1,5 @@\n 4.2\n+ * Print friendly err..."
...,...,...,...,...,...,...,...,...,...
64,6708499e4c930f7733894e6076b925fb151d9251,Remove metrics-reporter-config dependency\n\np...,12/9/2023,105,10,[CassandraDaemon.java],"[build-rat.xml, CHANGES.txt, NEWS.txt, jvm-ser...","[{'CassandraDaemon.java': '@@ -21,7 +21,6 @@ i...","[@@ -43,7 +43,6 @@\n <!-- Con..."
65,20d80118ac02d853d501e3f5c24a63a12d4010f5,Add the ability to disable bulk loading of SST...,19/9/2023,289,11,[Config.java],"[CHANGES.txt, NEWS.txt, cassandra.yaml, Config...","[{'Config.java': '@@ -860,6 +860,7 @@ public c...","[@@ -1,4 +1,5 @@\n 5.1\n+ * Add the ability to..."
66,d16e8d3653dce8ed767a040c06dbaabc47a9b474,Do not create sstable files before registering...,25/9/2023,724,18,[CompactionManager.java],"[CHANGES.txt, CompactionManager.java, Upgrader...","[{'CompactionManager.java': '@@ -1624,7 +1624,...","[@@ -1,4 +1,5 @@\n 5.0-alpha2\n+ * Do not crea..."
67,b9586501a6b6cdfe465302448018785652c9b966,Internode legacy SSL storage port certificate ...,25/9/2023,237,13,[MessagingService.java],"[CHANGES.txt, InboundConnectionInitiator.java,...","[{'MessagingService.java': '@@ -213,6 +213,7 @...","[@@ -1,4 +1,5 @@\n 4.1.4\n+ * Internode legacy..."


In [206]:
df_commits_with_critical_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   hash                       69 non-null     object
 1   msg                        69 non-null     object
 2   date                       69 non-null     object
 3   lines                      69 non-null     int64 
 4   files                      69 non-null     int64 
 5   critical_files             69 non-null     object
 6   modified_files             69 non-null     object
 7   diff_files                 69 non-null     object
 8   diff_files_modified_files  69 non-null     object
dtypes: int64(2), object(7)
memory usage: 5.0+ KB


# 2. Extrai issues de um issue tracker

## Configura ambiente para acessar Issue Tracker

In [209]:
# Initialize the Jira connection
print('Initialize the Jira connection')
jira = JIRA(JIRA_SERVER, basic_auth=(username, password))

# Create a JiraUtils instance
print('Create a JiraUtils instance')
jira_utils = JiraUtils(ISSUE_TRACKER_PROJECT, jira)

date1 = (2023, 1, 1)
date2 = (2023, 10, 4)
distance = 60
print('Define date intervals')
print(f'From: {str(date1)} to: {date2}, by: {distance} days of distance.')

Initialize the Jira connection
Create a JiraUtils instance
Define date intervals
From: (2023, 1, 1) to: (2023, 10, 4), by: 60 days of distance.


## Recupera os issues do período dado

In [210]:
# Fetch issues using date intervals
print('Fetch issues using date intervals')
block_of_issues = jira_utils.get_list_of_block_issues_by_dates(date1, date2, distance)

Fetch issues using date intervals
Aguarde...


Progress Message Analysis:  20%|██        | 1/5 [00:02<00:10,  2.72s/it]

Range: project=CASSANDRA and created>="2023/01/01" and created<="2023/03/01", qtd issues: 164


Progress Message Analysis:  40%|████      | 2/5 [00:05<00:07,  2.58s/it]

Range: project=CASSANDRA and created>="2023/03/02" and created<="2023/04/30", qtd issues: 194


...

Range: project=CASSANDRA and created>="2023/05/01" and created<="2023/06/29", qtd issues: 148


Progress Message Analysis:  80%|████████  | 4/5 [00:09<00:02,  2.37s/it]

Range: project=CASSANDRA and created>="2023/06/30" and created<="2023/08/28", qtd issues: 157


Progress Message Analysis: 100%|██████████| 5/5 [00:11<00:00,  2.40s/it]

Range: project=CASSANDRA and created>="2023/08/29" and created<="2023/10/27", qtd issues: 167
2023-11-02 15:21:32.287894
Tempo da consulta: 0:00:12.002747


In [211]:
# Concatenate the block of issues into a single list
print('Concatenate the block of issues into a single list')
all_issues = jira_utils.concatenate_block_of_issues(block_of_issues)

Concatenate the block of issues into a single list
Total de issues recuperados: 830


In [212]:
all_real_issues = analyze_jira_all_issues(ISSUE_TRACKER_PROJECT, all_issues)

Progress jira all issues analysis: 100%|██████████| 830/830 [00:00<00:00, 58468.49it/s]


### Converte os issues recuperados para um dataframe

In [213]:
df_all_reall_issues_in_commits_detailed = convert_issues_to_dataframe(all_real_issues)
df_all_reall_issues_in_commits_detailed

,issue_key,issue_type,status,priority,summary,description,comments
0,CASSANDRA-18292,Bug,Resolved,Normal,Gossip stateMapOrdering does not have correct ...,There is a bug when stateMapOrdering sees two ...,made 4.x with critical as I think that means 4...
1,CASSANDRA-18291,Bug,Resolved,Normal,Too early schema version change in system loca...,Schema version in the system local table is up...,https://github.com/apache/cassandra/pull/2192\...
2,CASSANDRA-18290,Bug,Open,Normal,Test Failure: SecondaryIndexTest.testUpdatesTo...,from\r\n- https://ci-cassandra.apache.org/job/...,I think I can take this ticket as I have some ...
3,CASSANDRA-18289,Bug,Resolved,Normal,Test Failure: sslnodetonode_test.TestNodeToNod...,from\r\n- https://ci-cassandra.apache.org/job/...,I haven't been able to reproduce this and it [...
4,CASSANDRA-18288,Bug,Resolved,Normal,Test Failure: TopPartitionsTest.basicRegularTo...,from\r\n- https://ci-cassandra.apache.org/job/...,Nothing on butler on this particular test meth...
...,...,...,...,...,...,...,...
825,CASSANDRA-18805,Task,Resolved,Normal,Upgrade caffeine to 3.1.8 and fix CIDR permiss...,3.1.8 is based on Java 11. This version is tes...,Version 3 comes with some breaking API changes...
826,CASSANDRA-18804,Bug,Patch Available,Normal,(Accord): Bug fixes from CASSANDRA-18675 to be...,This work is forking CASSANDRA-18675 and pulli...,
827,CASSANDRA-18803,Bug,Resolved,Normal,Refactor validation logic in StorageService.re...,This is a follow-up ticket of CASSANDRA-14319,[~aweisberg] would you mind to take a look? Su...
828,CASSANDRA-18802,Improvement,Triage Needed,Normal,Extend compaction interfaces to provide split ...,The current compaction interfaces allow a comp...,


### Recupera Issues com referência nos commits com classes críticas

In [214]:
dict_issues_in_commits, df_issues_in_commits_with_critical_classes = get_commits_with_critical_files_and_issues_in_this_commits(df_commits_with_critical_files)
df_issues_in_commits_with_critical_classes

,issue_key,issue_type,status,priority,summary,description,comments
9,CASSANDRA-18283,Improvement,Resolved,Normal,Enhance diagnostic nodetool tablestats output,The nodetool tablestats command lacks some ava...,"When it comes to sstable size, these metrics a..."
24,CASSANDRA-18268,Improvement,Resolved,Normal,Add more logging around CompactionManager oper...,Whenever a cleanup or garbagecollect operation...,[~milan_k] Thanks for picking this up so quick...
40,CASSANDRA-18252,Task,Resolved,Normal,Remove Scripted UDFs from trunk to enable easi...,CASSANDRA-17281 was created to do the work aro...,Leaving this as 5.x since the deprecation (CAS...
73,CASSANDRA-18219,Improvement,Resolved,Normal,Warning message on aggregation queries doesn't...,The existing aggregation query warning message...,Thank you for reporting this! I will take a cl...
75,CASSANDRA-18217,Improvement,Resolved,Normal,Allow CQL queries on multiple indexes without ...,The Index Group index was added by [CASSANDRA-...,It seems CASSANDRA-16092 have not check into t...
81,CASSANDRA-18211,Bug,Resolved,Normal,NoSuchFileException when removing a snapshot,Automatic deletion of expired snapshots mainta...,Thank you [~jlewandowski] for reporting this. ...
137,CASSANDRA-18155,Improvement,Resolved,Normal,Coordinator level metrics for read response an...,"We propose creating a new metric group, {{Clie...",|trunk|[patch|https://github.com/apache/cassan...
139,CASSANDRA-18153,Bug,Resolved,Normal,Memtable being flushed without hostId in versi...,On ticket CASSANDRA-16619 some files were chan...,CC [~jlewandowski] \nStorage service is not ye...
140,CASSANDRA-18152,Bug,Resolved,Normal,mockito-inline causes tests to fail beacause o...,"While working on CASSANDRA-14361, when we incl...",I was asked on the mailing list here (0) if th...
152,CASSANDRA-18140,Bug,Resolved,Normal,getsstables --show-levels JMX serialization error,While the interface is compliant and tested by...,[https://github.com/jrwest/cassandra/tree/jwes...


In [215]:
df_issues_in_commits_with_critical_classes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 9 to 827
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   issue_key    45 non-null     object
 1   issue_type   45 non-null     object
 2   status       45 non-null     object
 3   priority     45 non-null     object
 4   summary      45 non-null     object
 5   description  45 non-null     object
 6   comments     45 non-null     object
dtypes: object(7)
memory usage: 2.8+ KB


# Seleciona amostra dos Issues

In [216]:
confidence_level = 0.95
margin_of_error = 0.05
population_proportion = 0.8
population_size = len(df_issues_in_commits_with_critical_classes)

sample_size = calculate_sample_size(confidence_level, margin_of_error, population_proportion, population_size)
print(f'População de issues: {population_size}')
print(f"Sample size para inspeção: {sample_size}")

População de issues: 45
Sample size para inspeção: 38


In [217]:
sample_issues = select_issues_to_inspection(sample_size, df_issues_in_commits_with_critical_classes, my_date='02/11/2023')

38 para inspeção manual
Relação de Issues salvos em 02/11/2023 para inspeção.


In [218]:
generate_files_issues_to_inspection(sample_issues, df_issues_in_commits_with_critical_classes)

Analyzing issues: 100%|██████████| 45/45 [00:00<00:00, 999.57it/s]

File my_issues/CASSANDRA-18252 created with success!
File my_issues/CASSANDRA-18219 created with success!
File my_issues/CASSANDRA-18217 created with success!
File my_issues/CASSANDRA-18155 created with success!
File my_issues/CASSANDRA-18153 created with success!
File my_issues/CASSANDRA-18153 created with success!
File my_issues/CASSANDRA-18153 created with success!
File my_issues/CASSANDRA-18152 created with success!
File my_issues/CASSANDRA-18152 created with success!
File my_issues/CASSANDRA-18140 created with success!
File my_issues/CASSANDRA-18441 created with success!
File my_issues/CASSANDRA-18441 created with success!
File my_issues/CASSANDRA-18398 created with success!
File my_issues/CASSANDRA-18352 created with success!
File my_issues/CASSANDRA-18352 created with success!
File my_issues/CASSANDRA-18352 created with success!
File my_issues/CASSANDRA-18320 created with success!
File my_issues/CASSANDRA-18316 created with success!
File my_issues/CASSANDRA-18304 created with su

In [219]:
!zip -r my_issues.zip my_issues

updating: my_issues/ (stored 0%)
updating: my_issues/CASSANDRA-18352 (deflated 56%)
updating: my_issues/CASSANDRA-18592 (deflated 48%)
updating: my_issues/CASSANDRA-18153 (deflated 63%)
updating: my_issues/CASSANDRA-18152 (deflated 57%)
updating: my_issues/CASSANDRA-18155 (deflated 50%)
updating: my_issues/CASSANDRA-18749 (deflated 48%)
updating: my_issues/CASSANDRA-18697 (deflated 52%)
updating: my_issues/CASSANDRA-18217 (deflated 54%)
updating: my_issues/CASSANDRA-18219 (deflated 54%)
updating: my_issues/CASSANDRA-18730 (deflated 46%)
updating: my_issues/CASSANDRA-18304 (deflated 66%)
updating: my_issues/CASSANDRA-18555 (deflated 56%)
updating: my_issues/CASSANDRA-18140 (deflated 39%)
updating: my_issues/CASSANDRA-18298 (deflated 53%)
updating: my_issues/CASSANDRA-18615 (deflated 51%)
  adding: my_issues/CASSANDRA-18554 (deflated 51%)
  adding: my_issues/CASSANDRA-18504 (deflated 53%)
  adding: my_issues/CASSANDRA-18320 (deflated 59%)
  adding: my_issues/CASSANDRA-18398 (deflated 65%